<a href="https://colab.research.google.com/github/Rachhh53/ExploreWiki/blob/main/WikipediaApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs



In [1]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13476 sha256=13b581577a2af6b6b6119de7a546977ca659e0aef05a351868877950ef48cdc5
  Stored in directory: /root/.cache/pip/wheels/d3/24/56/58ba93cf78be162451144e7a9889603f437976ef1ae7013d04
Successfully built wikipedia-api


In [5]:
!pip install tqdm

In [1]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=24d9cbf35e683576373109714ce46248e895bf571fbf9ac778e00fe509bec8c2
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


# wikipedia-api

In [6]:
# https://towardsdatascience.com/auto-generated-knowledge-graphs-92ca99a81121
import wikipediaapi  # pip install wikipedia-api
import pandas as pd
import concurrent.futures
from tqdm import tqdm

In [3]:
def wiki_scrape(topic_name, verbose=True):
    def wiki_link(link):
        try:
            page = wiki_api.page(link)
            if page.exists():
                return {'page': link, 'text': page.text, 'link': page.fullurl,
                        'categories': list(page.categories.keys())}
        except:
            return None

    wiki_api = wikipediaapi.Wikipedia(language='en',
                                      extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(topic_name)
    if not page_name.exists():
        print('Page {} does not exist.'.format(topic_name))
        return

    page_links = list(page_name.links.keys())
    progress = tqdm(desc='Links Scraped', unit='', total=len(page_links)) if verbose else None
    sources = [{'page': topic_name, 'text': page_name.text, 'link': page_name.fullurl,
                'categories': list(page_name.categories.keys())}]

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_link = {executor.submit(wiki_link, link): link for link in page_links}
        for future in concurrent.futures.as_completed(future_link):
            data = future.result()
            sources.append(data) if data else None
            progress.update(1) if verbose else None
    progress.close() if verbose else None

    namespaces = ('Wikipedia', 'Special', 'Talk', 'LyricWiki', 'File', 'MediaWiki',
                  'Template', 'Help', 'User', 'Category talk', 'Portal talk')
    sources = pd.DataFrame(sources)
    sources = sources[(len(sources['text']) > 20)
                      & ~(sources['page'].str.startswith(namespaces, na=True))]
    sources['categories'] = sources.categories.apply(lambda x: [y[9:] for y in x])
    sources['topic'] = topic_name
    print('Wikipedia pages scraped:', len(sources))

    return sources

In [93]:
wiki_data = wiki_scrape('CVS Pharmacy')

Links Scraped: 100%|██████████| 233/233 [00:10<00:00, 21.30/s] 

Wikipedia pages scraped: 164


In [94]:
wiki_data

,page,text,link,categories,topic
0,CVS Pharmacy,"CVS Pharmacy, Inc. (styled CVSpharmacy, previo...",https://en.wikipedia.org/wiki/CVS_Pharmacy,"[1963 establishments in Massachusetts, All Wik...",CVS Pharmacy
1,20/20 (US television series),20/20 (stylized as 2020) is an American televi...,https://en.wikipedia.org/wiki/20/20_(American_...,"[1970s American television news shows, 1978 Am...",CVS Pharmacy
2,AEON (company),"ÆON Co., Ltd. (イオン株式会社, Ion kabushiki gaisha),...",https://en.wikipedia.org/wiki/AEON_(company),"[1758 establishments in Japan, Aeon Group, All...",CVS Pharmacy
3,Aetna,Aetna Inc. () is an American managed health c...,https://en.wikipedia.org/wiki/Aetna,"[1850 establishments in Connecticut, 2018 merg...",CVS Pharmacy
4,ABC News,ABC News is the news division of Walt Disney T...,https://en.wikipedia.org/wiki/ABC_News,"[ABC News, All articles needing additional ref...",CVS Pharmacy
...,...,...,...,...,...
159,Schwarz Gruppe,The Schwarz Group is a family-owned multinatio...,https://en.wikipedia.org/wiki/Schwarz_Gruppe,"[All stub articles, Articles with short descri...",CVS Pharmacy
160,Seven & i Holdings,"Seven & i Holdings Co., Ltd. (株式会社セブン&アイ・ホールディ...",https://en.wikipedia.org/wiki/Seven_%26_I_Hold...,"[Articles containing Japanese-language text, A...",CVS Pharmacy
161,"Southside Place, Texas",Southside Place is a city in west central Harr...,"https://en.wikipedia.org/wiki/Southside_Place,...","[Articles with LCCN identifiers, Articles with...",CVS Pharmacy
162,Smoking cessation,"Smoking cessation, usually called quitting smo...",https://en.wikipedia.org/wiki/Smoking_cessation,"[All Wikipedia articles in need of updating, A...",CVS Pharmacy


In [13]:
# Get Page Sections

def print_sections(sections, level=0):
        for s in sections:
                print("%s: %s - %s" % ("*" * (level + 1), s.title, s.text[0:40]))
                print_sections(s.sections, level + 1)

topic_name = 'Nestle'
wiki_api = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
page_name = wiki_api.page(topic_name)
print_sections(page_name.sections)

*: History - 
**: 1866–1900: Founding and early years - Nestlé's origin dates back to the 1860s,
***: Timeline - 1866, Charles Page (US consul to Switzer
**: 1901–1989: Mergers - In 1904, François-Louis Cailler, Charles
**: 1990–2011: Growth internationally - The first half of the 1990s proved to be
**: 2012–present: Recent developments - In recent years, Nestlé Health Science h
*: Corporate affairs and governance - Nestlé is the biggest food company in th
**: Financial data - 
**: Joint ventures - Joint ventures include:

Cereal Partners
**: Board of Directors - As of 2017, the board is composed of:
Pa
**: Lobbying - The company engages third party lobbying
*: Products - Nestlé currently has over 2,000 brands w
**: Food safety - 
***: Milk products and baby food - In late September 2008, the Hong Kong go
***: Cookie dough - In June 2009, an outbreak of E. coli O15
***: Maggi noodles - In May 2015, Food Safety Regulators from
*: Sponsorships - 
**: Music and Entertainment - In 1993, pl

# wptools - doesn't working on my machine

In [14]:
!pip install wptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 KB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycurl: filename=pycurl-7.45.1-cp37-cp37m-linux_x86_64.whl size=300958 sha256=deaf66084bc715eb9c9d7390b27450d9cf2a1a31b522783a8333517c4ef2b088
  Stored in directory: /root/.cache/pip/wheels/b4/fe/1f/e6cae4153dbc082271bd5763ad36ae255639e4641054db10bc
Successfully built pycurl


In [33]:
import wptools
so = wptools.page('Nestle').get_parse()
infobox = so.data['infobox']
infobox

en.wikipedia.org (parse) Nestle
en.wikipedia.org (imageinfo) File:NestleHQ.jpg
Nestlé (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:NestleHQ....
  infobox: <dict(24)> name, former_names, logo, image, image_size,...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:N...
  pageid: 160227
  parsetree: <str(193316)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Nestlé
  wikibase: Q160746
  wikidata_url: https://www.wikidata.org/wiki/Q160746
  wikitext: <str(164424)> {{Short description|Swiss multinational ...
}


{'name': 'Nestlé S.A.',
 'former_names': '{{collapsible list|\n* Anglo-Swiss Condensed Milk Company (1866–1867)\n* Farine Lactée Henri Nestlé (1867–1905)\n* Nestlé and Anglo-Swiss Condensed Milk Company (1905–1947)\n* Nestlé Alimentana SA (1947–)}}',
 'logo': 'Nestlé.svg',
 'image': 'NestleHQ.jpg',
 'image_size': '250px',
 'image_caption': 'Nestlé Headquarters in [[Vevey]], [[Vaud]], Switzerland',
 'type': '[[Public company|Public]]',
 'traded_as': '{{SWX|NESN}}',
 'ISIN': 'CH0038863350',
 'foundation': '{{start date and age|df|=|yes|1866}}',
 'founder': '[[Henri Nestlé]]',
 'location': '[[Vevey]], [[Vaud]], [[Switzerland]]',
 'area_served': 'Worldwide',
 'key_people': '[[Paul Bulcke]] <br> {{small|([[Chairman]])}} <br>[[Ulf Mark Schneider]] <br> {{small|([[CEO]])}} <br/>David McDaniel <br/> {{small|([[CFO]])}}',
 'industry': '[[Food processing]]',
 'products': '[[Baby food]], [[coffee]], [[dairy products]], [[breakfast cereals]], [[confectionery]], [[bottled water]], [[ice cream]], [[

# beautiful soup way

In [15]:
!pip install bs4
!pip install urllib3

In [24]:
# importing the module
import requests
import bs4

# URL
URL = "https://en.wikipedia.org/wiki/Twitter"

# sending the request
response = requests.get(URL)

# parsing the response
soup = bs4.BeautifulSoup(response.text, 'html')

# Now, we have parsed HTML with us. I want to get the _motto_ from the wikipedia page.
# Elements structure
# table - class="infobox"
# 3rd tr to get motto

# getting infobox
infobox = soup.find('table', {'class': 'infobox'})
infobox


# # getting 3rd row element tr
# third_tr = infobox.find_all('tr')[2]

# # from third_tr we have to find first 'a' element and 'div' element to get required data
# first_a = third_tr.div.find('a')
# div = third_tr.div.div

# # motto
# motto = f"{first_a.text} {div.text[:len(div.text) - 3]}"

# # printing the motto
# print(motto)


<table class="infobox vcard"><caption class="infobox-title fn org">Twitter, Inc.</caption><tbody><tr><td class="infobox-image" colspan="2"><a class="image" href="/wiki/File:Twitter-logo.svg"><img alt="Twitter-logo.svg" data-file-height="421" data-file-width="512" decoding="async" height="82" src="//upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Twitter-logo.svg/100px-Twitter-logo.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Twitter-logo.svg/150px-Twitter-logo.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Twitter-logo.svg/200px-Twitter-logo.svg.png 2x" width="100"/></a></td></tr><tr><td class="infobox-image" colspan="2"><div class="mw-collapsible mw-collapsed" style="box-sizing:border-box;width:100%;font-size:95%;padding:4px;border:none;"><div style="font-size:100%;line-height:1.6;font-weight:bold;background:transparent;text-align:left;;background:gainsboro;text-align:center">Screenshot</div><div class="mw-collapsible-content" style="backgr

In [19]:
from bs4 import BeautifulSoup
import urllib3
site= "http://en.wikipedia.org/wiki/Nestle"
hdr = {'User-Agent': 'Mozilla/5.0'}
http = urllib3.PoolManager()
req = http.request('GET', site)
#page = urllib3.urlopen(req)
soup = BeautifulSoup(req.data)
table = soup.find('table', class_='infobox vcard')
result = {}
exceptional_row_count = 0
for tr in table.find_all('tr'):
    if tr.find('th'):
        result[tr.find('th').text] = tr.find('td').text
    else:
        # the first row Logos fall here
        exceptional_row_count += 1
if exceptional_row_count > 1:
    print('WARNING ExceptionalRow>1: ', table)
print(result)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


WARNING ExceptionalRow>1:  <table class="infobox vcard"><caption class="infobox-title fn org" style="font-size: 125%;">Nestlé S.A.</caption><tbody><tr><td class="infobox-image logo" colspan="2"><a class="image" href="/wiki/File:Nestl%C3%A9.svg"><img alt="Nestlé.svg" data-file-height="86" data-file-width="300" decoding="async" height="63" src="//upload.wikimedia.org/wikipedia/en/thumb/d/d8/Nestl%C3%A9.svg/220px-Nestl%C3%A9.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/d/d8/Nestl%C3%A9.svg/330px-Nestl%C3%A9.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/d/d8/Nestl%C3%A9.svg/440px-Nestl%C3%A9.svg.png 2x" width="220"/></a></td></tr><tr><td class="infobox-image logo" colspan="2"><a class="image" href="/wiki/File:NestleHQ.jpg"><img alt="NestleHQ.jpg" data-file-height="595" data-file-width="800" decoding="async" height="186" src="//upload.wikimedia.org/wikipedia/commons/thumb/3/3e/NestleHQ.jpg/250px-NestleHQ.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/3/3

# Pandas html way ***

In [91]:
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/CVS_Pharmacy'
infoboxes = read_html(page, index_col=0, attrs={"class":"infobox"})
#wikitables = read_html(page, index_col=0, attrs={"class":"wikitable"})

print("Extracted {num} infoboxes".format(num=len(infoboxes)))
#print("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 infoboxes


In [58]:
wikitables[0]

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Year,,,,,,,,,,,
Revenue,83.642,92.186,92.158,91.612,88.785,89.469,89.791,91.439,92.568,84.343,87.088
Net income,9.487,10.611,10.015,14.456,9.066,8.531,7.183,10.135,12.609,12.232,17.196
Assets,114.091,126.229,120.442,133.450,123.992,131.901,130.380,137.015,127.940,124.028,139.142
Employees,328000.000,339000.000,333000.000,339000.000,335000.000,328000.000,323000.000,308000.000,291000.000,273000.000,276000.000


In [92]:
infobox_df = infoboxes[0]
infobox_df

,1
0,
NaN,NaN
Formerly,Consumer Value Stores (1963–69)
Type,Subsidiary
Industry,Retail
Founded,"May 8, 1963; 58 years agoLowell, Massachusetts..."
Founders,Stanley GoldsteinSidney GoldsteinRalph Hoaglan...
Headquarters,"1 CVS Drive, Woonsocket, Rhode Island, U.S."
Number of locations,"9,967[1] (2018)"
Area served,United States


In [63]:
founded = infobox_df.loc["Founded"]
industry = infobox_df.loc["Industry"]
num_employees = infobox_df.loc["Number of employees"]

In [65]:
founded

1    1866; 156 years ago
Name: Founded, dtype: object

In [66]:
industry

1    Food processing
Name: Industry, dtype: object

In [67]:
num_employees

1    276,000 (2021)[3]
Name: Number of employees, dtype: object

In [68]:
# find year founded
def get_founded(page):
    try:
        # get infoboxes
        infoboxes = read_html(page, index_col=0, attrs={"class": "infobox"})
        infobox_df = infoboxes[0]
        founded = infobox_df.loc["Founded"]
        return founded.to_string()
    except:
        return None

In [89]:
founded = get_founded('https://en.wikipedia.org/wiki/Verizon_Communications')
founded

1    October 7, 1983; 38 years ago
Name: Founded, dtype: object

In [88]:
founded.to_string()

'1    October\xa07, 1983; 38 years ago'

# wikipedia

In [2]:
import wikipedia

In [9]:
p = wikipedia.page('Nestle')
p.content

'Nestlé S.A. (; French: [nɛsle]) is a Swiss multinational food and drink processing conglomerate corporation headquartered in Vevey, Vaud, Switzerland. It is the largest publicly held food company in the world, measured by revenue and other metrics, since 2014. It ranked No. 64 on the Fortune Global 500 in 2017 and No. 33 in the 2016 edition of the Forbes Global 2000 list of largest public companies.Nestlé\'s products include baby food (some including human milk oligosaccharides), medical food, bottled water, breakfast cereals, coffee and tea, confectionery, dairy products, ice cream, frozen food, pet foods, and snacks. Twenty-nine of Nestlé\'s brands have annual sales of over 1 billion CHF (about US$1.1 billion), including Nespresso, Nescafé, Kit Kat, Smarties, Nesquik, Stouffer\'s, Vittel, and Maggi. Nestlé has 447 factories, operates in 189 countries, and employs around 339,000 people. It is one of the main shareholders of L\'Oreal, the world\'s largest cosmetics company.Nestlé was 